In [1]:
# importing libraries
import pandas as pd

In [2]:
# Analize submission data
# This is the data we will eventually predict prices for, therefore we will analize it to see the final conditions our model will find.
submission = pd.read_csv('../cars-competition/data/cars_test.csv', index_col='Id')
submission.head()

,city,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,...,type,paint_color,lat,long,county_fips,county_name,state_fips,state_code,state_name,weather
Id,,,,,,,,,,,,,,,,,,,,,
974298,duluth,2006.0,ford,f-250 super duty,good,8 cylinders,gas,154400.0,clean,automatic,...,pickup,white,47.746524,-90.357742,27031.0,Cook,27.0,MN,Minnesota,43.0
1051884,kansascity,1987.0,chevrolet,NaN,NaN,NaN,gas,NaN,clean,automatic,...,NaN,NaN,38.373182,-93.776859,29083.0,Henry/Rives,29.0,MO,Missouri,52.0
684464,palmsprings,2010.0,jeep,liberty sport,NaN,6 cylinders,gas,127722.0,clean,automatic,...,SUV,NaN,33.741059,-116.356434,6065.0,Riverside,6.0,CA,California,59.0
1255387,sanmarcos,2003.0,chevrolet,tahoe,fair,8 cylinders,gas,NaN,clean,automatic,...,SUV,white,30.026266,-98.133363,48209.0,Hays,48.0,TX,Texas,67.0
1195520,tampa,2006.0,lexus,gs 300,NaN,NaN,gas,NaN,clean,automatic,...,NaN,NaN,27.868800,-82.734400,12103.0,Pinellas,12.0,FL,Florida,65.0


In [3]:
# Checking for Null values
print(submission.shape)
print(submission.isnull().sum())

(253073, 22)
city                 0
year               911
manufacturer     20048
make             10112
condition       102695
cylinders       101146
fuel              1583
odometer         83110
title_status       373
transmission      1350
drive            96866
size            164839
type            103023
paint_color     101787
lat                  0
long                 0
county_fips       8632
county_name       8632
state_fips        8632
state_code        8632
state_name           0
weather           8716
dtype: int64


In [4]:
# Percentage of null values
print(100*submission.isnull().sum()/submission.shape[0])

city             0.000000
year             0.359975
manufacturer     7.921825
make             3.995685
condition       40.579200
cylinders       39.967124
fuel             0.625511
odometer        32.840327
title_status     0.147388
transmission     0.533443
drive           38.275912
size            65.134961
type            40.708807
paint_color     40.220411
lat              0.000000
long             0.000000
county_fips      3.410874
county_name      3.410874
state_fips       3.410874
state_code       3.410874
state_name       0.000000
weather          3.444066
dtype: float64


In [5]:
# Quite a few relevant values have many null values: condition, cylinders, odometer, drive, size, type
# We will have to try to find a way around it.

In [6]:
# Cylinders
# Electric cars will have null values, will set those to zero.
# Change nulls to other
submission['cylinders'] = submission['cylinders'].str.replace(' cylinders', '')

null_cylinders = submission[submission['cylinders'].isnull()]
for index, car in null_cylinders.iterrows():
    if car['fuel'] == 'electric':
        submission.at[index,'cylinders'] = 0
    else:
        submission.at[index,'cylinders'] = 'other'

In [7]:
submission['cylinders'].isnull().sum()

0

In [8]:
not_null_cylinders = submission[submission['cylinders'] != 'other']['cylinders']
not_null_cylinders.isnull().sum()

0

In [9]:
# Check mean and median of cylinders
# Change to numeric
not_null_cylinders = not_null_cylinders.apply(int)
mean = not_null_cylinders.mean()
median = not_null_cylinders.median()
mean, median

(6.060791638597995, 6.0)

In [10]:
# Change 'other' for median
submission['cylinders'][submission['cylinders']=='other'] = median
submission['cylinders'] = submission['cylinders'].astype('int64')
submission['cylinders'].value_counts()

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


6     159533
8      47329
4      43131
5       1577
10       894
3        267
0        226
12       116
Name: cylinders, dtype: int64

In [11]:
# Condition
# Will change to numeric and convert nulls to central number
submission['condition'][submission['condition'].isnull()] = 'unknown'
display(submission['condition'].value_counts())
display(submission['condition'].isnull().sum())

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


unknown      102695
excellent     64990
good          55911
like new      16342
fair          10935
new            1301
salvage         899
Name: condition, dtype: int64

0

In [12]:
condition = {'unknown':0,'fair':0,'good':1,'excellent':2,'like new':2,'new':2,'salvage':-1}
submission['condition']=submission['condition'].apply(lambda x: condition.get(x,0))

In [13]:
submission['condition'].value_counts()

 0    113630
 2     82633
 1     55911
-1       899
Name: condition, dtype: int64

In [14]:
# Odometer
# Will check mean and median by year

# First, clean year.
display(submission['year'].value_counts())
display(submission['year'].isnull().sum())

# Replace unknown and wrong years by average
valid_year = submission['year'][((submission['year']>=1900) | (~submission['year'].isnull()))]
year_avg = valid_year.mean()
year_median = valid_year.median()
year_mode = valid_year.mode()
print(year_avg,year_median,year_mode)
submission['year'][((submission['year']<1900) | (submission['year'].isnull()))] = year_avg

2007.0    16041
2008.0    15064
2006.0    14754
2015.0    13895
2005.0    13362
          ...  
1913.0        1
1906.0        1
1740.0        1
302.0         1
1553.0        1
Name: year, Length: 112, dtype: int64

911

2004.795575859963 2007.0 0    2007.0
dtype: float64


/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
# Replace Null odometer by mean of decade or absolute mean
submission['decade'] = submission['year']//10
null_odometer = submission[submission['odometer'].isnull()]
odometer_by_decade_mean = submission[['decade','odometer']].groupby(by='decade').mean()
odometer_by_decade_median = submission[['decade','odometer']].groupby(by='decade').median()
odometer_mean = submission['odometer'].mean()

for index, car in null_odometer.iterrows():
    submission.at[index,'odometer'] = odometer_by_decade_mean.get(car['decade'],odometer_mean)

In [17]:
# Drive
# Replace unknown with fwd, as it is the most comon drive
display(submission['drive'].value_counts())
display(submission['drive'].isnull().sum())
submission['drive'][submission['drive'].isnull()]='fwd'

4wd    66050
fwd    54350
rwd    35807
Name: drive, dtype: int64

96866

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [73]:
# Size
# Replace unknown with most common by cylinders
null_size = submission[submission['size'].isnull()]
size_by_cylinder = submission[['size','cylinders']].groupby(by='cylinders').agg(lambda x:x.value_counts().index[0])
size_by_cylinder = {cyl:size for cyl,size in zip(size_by_cylinder.index,size_by_cylinder['size'])}
for index, car in null_size.iterrows():
    submission.at[index,'size'] = size_by_cylinder.get(car['cylinders'],'other')
size = {'sub-compact':0,'compact':1,'mid-size':2,'full-size':3}
submission['size'].value_counts()

full-size      191034
compact         34289
mid-size        25936
sub-compact      1814
Name: size, dtype: int64

In [58]:
# Manufacturer
# Fix multiple values
manufacturer = {'chevy':'chevrolet','chev':'chevrolet','vw':'volkswagen','mercedes':'mercedes-benz','mercedesbenz':'mercedes-benz','infinity':'infiniti','harley':'harley-davidson','alfa':'alfa-romeo','aston':'aston-martin','landrover':'land rover','rover':'land rover'}
wrong_manufacturers = submission[submission['manufacturer'].isin(manufacturer.keys())]
for index, car in wrong_manufacturers.iterrows():
    submission.at[index,'manufacturer'] = manufacturer.get(car['manufacturer'],'unknown')
# Replace null with unknown
submission['manufacturer'][submission['manufacturer'].isnull()] = 'unknown'

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [63]:
# Fuel
# Replace Null with 'other'
submission['fuel'][submission['fuel'].isnull()]='other'
submission['fuel'].value_counts()

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


gas         224804
diesel       17870
other         8505
hybrid        1578
electric       316
Name: fuel, dtype: int64

In [67]:
# Transmission
submission['transmission'][submission['transmission'].isnull()]='other'
submission['transmission'].value_counts()

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


automatic    216458
manual        29969
other          6646
Name: transmission, dtype: int64

In [70]:
# Title Status
submission['title_status'][submission['title_status'].isnull()]='clean'
submission['title_status'].value_counts()

/home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


clean         235668
rebuilt         7156
salvage         4639
lien            3242
missing         1622
parts only       746
Name: title_status, dtype: int64

In [71]:
print(100*submission.isnull().sum()/submission.shape[0])

city             0.000000
year             0.000000
manufacturer     0.000000
make             3.995685
condition        0.000000
cylinders        0.000000
fuel             0.000000
odometer         0.000000
title_status     0.000000
transmission     0.000000
drive            0.000000
size             0.000000
type            40.708807
paint_color     40.220411
lat              0.000000
long             0.000000
county_fips      3.410874
county_name      3.410874
state_fips       3.410874
state_code       3.410874
state_name       0.000000
weather          3.444066
decade           0.000000
dtype: float64


In [72]:
submission.dtypes

city             object
year            float64
manufacturer     object
make             object
condition         int64
cylinders         int64
fuel             object
odometer        float64
title_status     object
transmission     object
drive            object
size             object
type             object
paint_color      object
lat             float64
long            float64
county_fips     float64
county_name      object
state_fips      float64
state_code       object
state_name       object
weather         float64
decade          float64
dtype: object